In [1]:
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import autocorrelation_plot
sns.set_style("dark")

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pandas_datareader.data import DataReader
import datetime as dt


# Time series
from statsmodels.tsa.seasonal import seasonal_decompose

#metrics
from sklearn.metrics import mean_squared_error

# custom functions
import functions

In [2]:
# Important dates
start = dt.datetime(2017,8,6)
end = dt.datetime(2022,8,6)

In [3]:
# import data
nflx = DataReader("NFLX",  "yahoo", start, end)
gspc = DataReader("^GSPC", "yahoo", start, end)

In [4]:
# concatenate stock and market data
features = concatRename(nflx, 'nflx_', gspc, 'gspc_')

NameError: name 'concatRename' is not defined

In [ ]:
# remove test data from train
train, test = testSplit(features)

In [ ]:
def

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1008 entries, 2017-08-07 to 2021-08-06
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nflx_nflx_High       1008 non-null   float64
 1   nflx_nflx_Low        1008 non-null   float64
 2   nflx_nflx_Open       1008 non-null   float64
 3   nflx_nflx_Close      1008 non-null   float64
 4   nflx_nflx_Volume     1008 non-null   int64  
 5   nflx_nflx_Adj Close  1008 non-null   float64
 6   gspc_gspc_High       1008 non-null   float64
 7   gspc_gspc_Low        1008 non-null   float64
 8   gspc_gspc_Open       1008 non-null   float64
 9   gspc_gspc_Close      1008 non-null   float64
 10  gspc_gspc_Volume     1008 non-null   int64  
 11  gspc_gspc_Adj Close  1008 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 102.4 KB
